In [1]:
import sys
sys.path.insert(0,r"C:\Users\pc\Documents")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
## Lectura de datos     
base_costa = pd.read_csv(r'C:\Users\pc\Downloads\BASE_COSTA.txt', sep=r'¡', engine='python')
base_costa.head()

## Eliminación de variables: 
base_costa2 = base_costa[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO',
        'PROM_SUSC', 'CANAL_ENVIO', 'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

In [3]:
# Vista previa
base_costa2.head()

,SESUSERV,SERVDESC,SESUSUSC,SESUFEIN,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,SUSCNITC,TIPIFICACION_CLIENTE,...,CANTIDAD,CALIFICACION_VECTOR_SERVICIO,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,REGIONAL,ESTRATO_AGRUPADO,DEPARTAMENTO_AGR,ANTIGUEDAD_DIAS,CANAL_PAGO
0,7255,TELEFONÍA,17081442,19/01/2020,ATLANTICO,2,60522,7,1001999449,NUEVO,...,3,47,38726,DIGITAL,Hogares,COSTA,1.0,ATLANTICO,121,DIGITAL
1,3905,TELEVISIÓN,17085543,21/01/2020,ATLANTICO,2,66976,7,1002234238,NUEVO,...,2,38,26021,FISICO,Hogares,COSTA,4.0,ATLANTICO,119,FISICO
2,7255,TELEFONÍA,17034232,24/12/2019,ATLANTICO,3,53747,7,1001871120,NUEVO,...,4,16,"28961,5",DIGITAL,Hogares,COSTA,1.0,ATLANTICO,147,FISICO
3,7245,INTERNET,17021490,17/12/2019,BOLIVAR,2,94626,7,1143412558,NUEVO,...,3,67,"28330,2",DIGITAL,Hogares,COSTA,2.0,BOLIVAR,154,FISICO
4,7245,INTERNET,17068372,13/01/2020,SUCRE,2,82814,7,1102831117,NUEVO,...,2,33,"25499,83",DIGITAL,Hogares,COSTA,4.0,SUCRE,127,FISICO


In [4]:
## Numero completo de clientes en la region
len(base_costa2)

769688

In [5]:
## Eliminacion de clientes con datos NA
base_costa3 = base_costa2.dropna()

In [6]:
len(base_costa3)

733865

In [7]:
## Group by por nit para calificacion
b4 = base_costa3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_costa4 = pd.merge(base_costa3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_costa4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_costa4['y'] = pd.cut(x=base_costa4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])


In [10]:
dataTypeSeries = base_costa4.dtypes
dataTypeSeries

cat = base_costa4.select_dtypes(include=['object','category'])
cuanti = base_costa4.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))

In [15]:
cuanti.corr(method = 'pearson')

,SESUSERV,SESUSUSC,SESUCUSA,SESUCICL,SUMA,CANTIDAD,CALIFICACION_VECTOR_SERVICIO_x,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CALIFICACION_VECTOR_SERVICIO_y
SESUSERV,1.000000,0.613458,-0.032987,0.358026,-0.047300,-0.030081,-0.063268,0.184989,-0.078466,-0.062566
SESUSUSC,0.613458,1.000000,-0.018427,0.241125,-0.210372,-0.204276,-0.152834,0.275080,-0.247418,-0.151454
SESUCUSA,-0.032987,-0.018427,1.000000,-0.078362,-0.380746,-0.088775,-0.479242,-0.072959,0.054760,-0.471407
SESUCICL,0.358026,0.241125,-0.078362,1.000000,0.349398,0.373239,0.165170,0.081432,0.070978,0.171438
SUMA,-0.047300,-0.210372,-0.380746,0.349398,1.000000,0.895369,0.747490,0.019615,0.104153,0.731578
CANTIDAD,-0.030081,-0.204276,-0.088775,0.373239,0.895369,1.000000,0.529183,0.011295,0.126231,0.513706
CALIFICACION_VECTOR_SERVICIO_x,-0.063268,-0.152834,-0.479242,0.165170,0.747490,0.529183,1.000000,0.023038,0.055964,0.965152
ESTRATO_AGRUPADO,0.184989,0.275080,-0.072959,0.081432,0.019615,0.011295,0.023038,1.000000,-0.374972,0.023869
ANTIGUEDAD_DIAS,-0.078466,-0.247418,0.054760,0.070978,0.104153,0.126231,0.055964,-0.374972,1.000000,0.057985
CALIFICACION_VECTOR_SERVICIO_y,-0.062566,-0.151454,-0.471407,0.171438,0.731578,0.513706,0.965152,0.023869,0.057985,1.000000


In [14]:
base_costa5=cuanti.join(cat)
print(base_costa5.dtypes)

base_costa5.to_csv(r'C:\Users\pc\Desktop\base_sur_process.csv')

#base_costa5 = pd.read_csv(r'C:\Users\pc\Desktop\base_sur_process.csv', sep=',', engine='python')

X = base_costa5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 
        #'SESUSUSC',
        'DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12',
        #'REGIONAL',
        'PROM_SUSC', 'CANAL_ENVIO',
        # 'SUMA',
        #'CANTIDAD',
        #'CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y','DEPARTAMENTO_AGR'
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
       'SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

y = base_costa5['y']

SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUCICL                            int64
SUMA                                int64
CANTIDAD                            int64
CALIFICACION_VECTOR_SERVICIO_x      int64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                     int64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SESUSAPE                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
PROM_SUSC                           int32
CANAL_ENVIO                         int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [16]:
## Vista previa de las variables preliminares para clasificacion
X.head()

,SESUSERV,SERVDESC,DEPADESC,SESUCUSA,SESUSAPE,SESUCICL,TIPIFICACION_CLIENTE,PROM_SUSC,CANAL_ENVIO,PROM_SUSC,CANAL_ENVIO,SEGMENTACION,ESTRATO_AGRUPADO,ANTIGUEDAD_DIAS,CANAL_PAGO
0,7255,28,0,2,79992,7,1,133493,0,133493,0,1,1.0,121,0
1,3905,30,0,2,85379,7,1,39513,1,39513,1,1,4.0,119,1
2,7255,28,0,3,74462,7,1,57439,0,57439,0,1,1.0,147,1
3,7245,8,1,2,106977,7,1,52889,0,52889,0,1,2.0,154,1
4,7245,8,7,2,97677,7,1,36605,0,36605,0,1,4.0,127,1


In [28]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.833179  0.831980  0.832579   31288.0   31243.0  0.944545
1             0.800806  0.826609  0.813503   62754.0   64776.0  0.893511
2             0.724613  0.697446  0.710770   52731.0   50754.0  0.838271
avg / total   0.780333  0.781349  0.780660  146773.0  146773.0  0.899367
DecisionTreeClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.935817  0.753068  0.834555   31288.0   25178.0  0.948254
1             0.806884  0.786324  0.796472   62754.0   61155.0  0.894558
2             0.684828  0.784946  0.731477   52731.0   60440.0  0.846968
avg / total   0.790518  0.778740  0.781240  146773.0  146773.0  0.916458
RandomForestClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.751287  0.723025  0.736885 

In [29]:
# Importancia de variables
model = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)

importance = model.feature_importances_
# Total importancia de variables
for i,v in enumerate(importance):
	print('Variable: %0d, Score: %.5f' % (i,v))

Variable: 0, Score: 0.00134
Variable: 1, Score: 0.00000
Variable: 2, Score: 0.00000
Variable: 3, Score: 0.43030
Variable: 4, Score: 0.10111
Variable: 5, Score: 0.03169
Variable: 6, Score: 0.15977
Variable: 7, Score: 0.00015
Variable: 8, Score: 0.00000
Variable: 9, Score: 0.00002
Variable: 10, Score: 0.00000
Variable: 11, Score: 0.02775
Variable: 12, Score: 0.00000
Variable: 13, Score: 0.11718
Variable: 14, Score: 0.13068


In [ ]:
## El mejor modelo de clasificacion fue el KNN, veamos el k optimo

In [30]:
classifiers = [
    KNeighborsClassifier(2),
    KNeighborsClassifier(4),
    KNeighborsClassifier(5),
    ]

# Competencia de clasificadores
log_cols=["Classifier", "Accuracy", "Log Loss", "F1_SCORE"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__  
    print("="*30)
    print(name)
    print('****Resultados****')    
    train_predictions = clf.predict_proba(X_test)
    acc = cr.class_report(y_true = y_test, 
                                y_pred = clf.predict(X_test),
                                y_score = train_predictions)
    print("Métricas: {}".format(acc))
    log_entry = pd.DataFrame([[name, acc]])
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.774501  0.893889  0.829923   31288.0   36111.0  0.930307
1             0.744526  0.893553  0.812261   62754.0   75315.0  0.875826
2             0.794127  0.532324  0.637390   52731.0   35347.0  0.818300
avg / total   0.768736  0.763846  0.753200  146773.0  146773.0  0.880432
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.828623  0.858284  0.843193   31288.0   32408.0  0.951203
1             0.771847  0.882063  0.823283   62754.0   71715.0  0.902218
2             0.771043  0.623637  0.689550   52731.0   42650.0  0.848855
avg / total   0.783661  0.784150  0.779481  146773.0  146773.0  0.908812
KNeighborsClassifier
****Resultados****
Métricas:              precision    recall  f1-score   support      pred       AUC
0             0.859026  0.826930  0.842673   31

In [33]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
#X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
#'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_fin = X
X_ = StandardScaler().fit_transform(X_fin)

for train_index, test_index in kf.split(X_, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(5).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                      y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


[             precision    recall  f1-score   support      pred       AUC
0             0.696118  0.732026  0.713621   31309.0   32924.0  0.896470
1             0.719898  0.743687  0.731599   62732.0   64805.0  0.836578
2             0.601721  0.559639  0.579917   52734.0   49046.0  0.744934
avg / total   0.672366  0.675074  0.673267  146775.0  146775.0  0.837718,              precision    recall  f1-score   support      pred       AUC
0             0.692966  0.698234  0.695590   31309.0   31547.0  0.880359
1             0.719550  0.753650  0.736205   62732.0   65705.0  0.836184
2             0.602136  0.565471  0.583228   52733.0   49522.0  0.742854
avg / total   0.671695  0.674220  0.672580  146774.0  146774.0  0.832990,              precision    recall  f1-score   support      pred       AUC
0             0.692240  0.704357  0.698246   31308.0   31856.0  0.884081
1             0.725921  0.750060  0.737793   62731.0   64817.0  0.843936
2             0.605980  0.575712  0.590458   527

In [38]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

array([[22388,  2457,  6463],
       [ 2795, 46893, 13043],
       [ 7917, 15124, 29692]], dtype=int64)